In [2]:
# some important libraries
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# for video processing
import opyf
import sys
import os
# for Observations
import requests, json
from pytz import timezone 
from datetime import datetime
# for sending data via email 
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import PIL
import io

In [3]:
# some important functions
def GoalSeek(fun,goal,x0,fTol=0.0001,MaxIter=1000):
    if fun(x0)==goal:
        print('Exact solution found')
        return x0

    # Line Search Method
    step_sizes=np.logspace(-1,4,6)
    scopes=np.logspace(1,5,5)

    vFun=np.vectorize(fun)
    
    for scope in scopes:
        break_nested=False
        for step_size in step_sizes:

            cApos=np.linspace(x0,x0+step_size*scope,int(scope))
            cAneg=np.linspace(x0,x0-step_size*scope,int(scope))

            cA=np.concatenate((cAneg[::-1],cApos[1:]),axis=0)

            fA=vFun(cA)-goal

            if np.any(np.diff(np.sign(fA))):

                index_lb=np.nonzero(np.diff(np.sign(fA)))

                if len(index_lb[0])==1:

                    index_ub=index_lb+np.array([1])

                    x_lb=np.asscalar(np.array(cA)[index_lb][0])
                    x_ub=np.asscalar(np.array(cA)[index_ub][0])
                    break_nested=True
                    break
                else: # Two or more roots possible

                    index_ub=index_lb+np.array([1])

                    print('Other solution possible at around, x0 = ', np.array(cA)[index_lb[0][1]])

                    x_lb=np.asscalar(np.array(cA)[index_lb[0][0]])
                    x_ub=np.asscalar(np.array(cA)[index_ub[0][0]])
                    break_nested=True
                    break

        if break_nested:
            break
    if not x_lb or not x_ub:
        print('No Solution Found')
        return

    # Bisection Method
    iter_num=0
    error=10

    while iter_num<MaxIter and fTol<error:
        
        x_m=(x_lb+x_ub)/2
        f_m=fun(x_m)-goal

        error=abs(f_m)

        if (fun(x_lb)-goal)*(f_m)<0:
            x_ub=x_m
        elif (fun(x_ub)-goal)*(f_m)<0:
            x_lb=x_m
        elif f_m==0:
            print('Exact spolution found')
            return x_m
        else:
            print('Failure in Bisection Method')
        
        iter_num+=1

    return x_m

def df_to_array ( path ): # set in initialVelocitydf
    # Determine the dimensions of the 2D array based on the maximum X and Y coordinates
    df = pd.read_csv ( path )
    
    max_x = int (df['X'].iloc[-1])
    max_y = int (df['Y'].iloc[-1])
    array_shape = (max_y + 2, max_x + 2, 2)
    
    # Create an empty 2D NumPy array
    array = np.empty(array_shape)
    
    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        x = int (row['X'])
        y = int (row['Y'])
        ux = row['Ux_[px.deltaT^{-1}]']
        uy = row['Uy_[px.deltaT^{-1}]']

        # Assign the velocity vector to the corresponding element in the array
        array[y, x] = np.array ([ux, uy])
        array[y+1, x] = np.array ([ux, uy])
        array[y, x+1] = np.array ([ux, uy])
        array[y+1, x+1] = np.array ([ux, uy])

    return array

def ImageVelocity ( path ): # returns a numpy array with same dimensions as our original video
    # give code here,
    video=opyf.videoAnalyzer(path)
    video.set_vecTime(Ntot=10,starting_frame=100)
    
    video.set_interpolationParams(Sharpness=2)
    video.set_goodFeaturesToTrackParams(qualityLevel=0.01)
    
    video.set_vlim([0, 10])
    
    video.extractGoodFeaturesDisplacementsAccumulateAndInterpolate(display1='quiver',display2='field',displayColor=True)
    video.set_filtersParams(maxDevInRadius=1.5, RadiusF=0.15,range_Vx=[0.01,10])
    
    video.filterAndInterpolate()

    video.writeVelocityField(fileFormat='csv')
    name = "frame_100_to_110_with_step_1_and_shift_1.csv"
    
    velocityArray = df_to_array ( path = name )
    return velocityArray

def DistanceAdjust ( velocityArray, f_d, xl, xr, yp, width, height, Z ): # returns corrected values of the velocities along yp
    # given the velocity array, the width, the height, and xl, xr, yp, 
    # to get the corrected values in an array
    
    center_x = width // 2
    center_y = height // 2
    
    disparity = []
    target_velocities = []
    
    for x in range(xl, xr+1):
        velocity = velocityArray[yp, x]
        target_velocities.append(velocity)
        
        dx = x - center_x
        dy = yp - center_y
        distance = np.sqrt(dx**2 + dy**2)
        disparity.append(distance)
        
    disparity = np.array(disparity)
    target_velocities = np.array(target_velocities)
    
    # now, to get the corresponding widths associated with each targeted pixel
    
    L = Z * np.tan ( disparity/f_d )
    
    Z_for_pixels = ( L**2 + Z**2 ) ** 0.5
    Z_for_pixels = np.repeat(Z_for_pixels[np.newaxis, :], 2, axis=0)
    Z_for_pixels = Z_for_pixels.transpose()
    
    true_velocities = target_velocities * Z_for_pixels
    return true_velocities

def save_first_frame_as_numpy_array(path):
    # Read the video using OpenCV
    video = cv2.VideoCapture(path)
    
    # Check if the video file was successfully opened
    if not video.isOpened():
        raise ValueError(f"Could not open the video file: {path}")
    
    # Read the first frame
    success, frame = video.read()

    # Check if a frame was successfully read
    if not success:
        raise ValueError(f"Could not read the first frame of the video: {path}")

    # Convert the frame to a NumPy array
    frame_array = np.array(frame)

    # Release the video capture object
    video.release()

    return frame_array

In [4]:
def initial_alignment (  path, Z, stream_width  ):
    cap = cv2.VideoCapture(path)
    play = True
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    xl , xr = 0 , width
    yp = height // 2
    
    while play:
        while cap.isOpened():

            key = cv2.waitKey(1) & 0xFF
            ret, frame = cap.read()

            if not ret:
                break
                
            # Draw lines on the frame
            cv2.line(frame, (xl, 0), (xl, frame.shape[0]), (0, 255, 0), 2)  # Vertical line at xl
            cv2.line(frame, (xr, 0), (xr, frame.shape[0]), (0, 255, 0), 2)  # Vertical line at xr
            cv2.line(frame, (0, yp), (frame.shape[1], yp), (0, 255, 0), 2)  # Horizontal line at yp
            
            cv2.imshow('Video', frame)
            if key == ord('q'):
                play = False
                plt.imshow ( frame )
                plt.show ()
                break
            elif key == ord('d'):  # Increase xl by 1 (right arrow key)
                xl += 1
            elif key == ord('a'):  # Decrease xr by 1 (left arrow key)
                xr -= 1
            elif key == ord('w'):  # Decrease yp by 1 (up arrow key)
                yp -= 1
            elif key == ord('s'):  # Increase yp by 1 (down arrow key)
                yp += 1

        cap.release()
        cap = cv2.VideoCapture(path)

    cap.release()
    cv2.destroyAllWindows()

    print ( f"Left bank is at {xl}" )
    print ( f"Right bank is at {xr}" )
    print ( f"Line for measurement is at {yp}" )
    
    #----
    # to get the f_d ratio
    f_d = (xr-xl)/2 / np.arctan (stream_width/2/Z)
    
    return xl , xr , yp , width , height , f_d

In [5]:
def processing_and_discharge_calculation ( path, xl , xr , yp , width , height , f_d ):
    #----
    # now, use video in our functions.
    velocityArray = ImageVelocity (path = path)
    velocityArrayRequired = DistanceAdjust ( velocityArray, f_d, xl, xr, yp, width, height, Z )
    
    Velocity = pd.Series ( velocityArrayRequired[:,1] )
    
    # now, calculate the discharge,

    def generate_D(x):
        something = np.abs(np.random.normal(0.8,0.5,int((x+1)/2)))
        something = np.concatenate((np.sort(something),np.sort(something)[::-1]),axis = None)
        if(len(something)>x):
            something = something[:-1]
        return something
    
    U_surface = np.absolute(Velocity.values)
    data_len = len(U_surface)
    x = np.linspace(0.1,5,len(U_surface))
    # D = np.linspace(0.1,5,len(U_surface))
    D = generate_D(data_len)
    y = np.linspace(0.1,1,100)

    area_of_one_point = 5 / (data_len * 100)

    print(x[0:5])
    print(area_of_one_point)
    print(D[0:5])

    phi_Mobs = np.mean(U_surface) / np.max(U_surface)

    U_max = np.max(U_surface)
    U_mean = np.mean(U_surface)

    def find_M():
        def fun(x):
            y = np.exp(x) / (np.exp(x)-1)
            y = y - 1/x
            return y

        goal=U_mean / U_max

        # (iii) Define a starting point
        x0=0.001

        ## Here is the result
        ans = GoalSeek(fun,goal,x0)
    #     print('M is = ', ans)
        return ans

    def implement_flow_chart():
        p = 1
        a = 0.05
        u_xy = []

        delta_dash = a + 1 + 1.3 * np.exp(-x/D)
        M = find_M()

        # h missing in flow-chart
        h = D - D/delta_dash

        u_maxv = U_surface / (1/M * np.log(1 + (np.exp(M)-1)* delta_dash * np.exp(1-delta_dash)))

        for j in range(len(y)):
            temp = u_maxv/M * np.log(1+ np.absolute( (np.exp(M) -1)*y[j]*np.exp(1- y[j]/(D-h))) )
            u_xy.append(temp)

        phi_Mp = np.mean(u_maxv) / np.max(u_maxv)

        p = p+1
        a = a + 1

        while not(phi_Mp-phi_Mobs <0.03):
            delta_dash = a + 1 + 1.3 * np.exp(-x/D)
            M = find_M()

            # h missing in flow-chart
            h = D - D/delta_dash

            u_maxv = U_surface / (1/M * np.log(1 + (np.exp(M)-1)* delta_dash * np.exp(1-delta_dash)))
            print("Just before for loop")

            for j in range(len(y)):
    #             print (1+ (np.exp(M) -1)*y[j]*np.exp(1- y[j]/(D-h)))
    #             print("1st term",np.exp(M) -1)
    #             print("2nd term",y[j])
                temp = u_maxv/M * np.log(1+ np.absolute( (np.exp(M) -1)*y[j]*np.exp(1- y[j]/(D-h)) ) )
                u_xy.append(temp)

            phi_Mp = np.mean(u_maxv) / np.max(u_maxv)

            p = p+1
            a = a + 1

        u_xy = np.array(u_xy)
        return u_xy

    ans = implement_flow_chart()
    ans = ans * area_of_one_point
    discharge = np.sum(ans)
    discharge = abs (round( discharge,4 ))
    print( "Final discharge of the river",discharge,"m^3/sec")
    
    return discharge

In [6]:
def observation_get ( video_path, save_path, discharge, state_name, city_name ):

    # Enter your API key here
    api_key = "e23dc600314fe22e1fee382f851a706c"

    # base_url variable to store url
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # complete_url variable to store
    # complete url address
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name

    # get method of requests module
    # return response object
    response = requests.get(complete_url)

    # json method of response object
    # convert json format data into
    # python format data
    x = response.json()

    # Now x contains list of nested dictionaries
    # Check the value of "cod" key is equal to
    # "404", means city is found otherwise,
    # city is not found
    if x["cod"] != "404":

        # store the value of "main"
        # key in variable y
        y = x["main"]

        # store the value corresponding
        # to the "temp" key of y
        current_temperature = y["temp"]

        # store the value corresponding
        # to the "pressure" key of y
        current_pressure = y["pressure"]

        # store the value corresponding
        # to the "humidity" key of y
        current_humidity = y["humidity"]

        # store the value of "weather"
        # key in variable z
        z = x["weather"]

        # store the value corresponding
        # to the "description" key at
        # the 0th index of z
        weather_description = z[0]["description"]

        # print following values
        print(" Temperature (in kelvin unit) = " +
                        str(current_temperature) +
            "\n atmospheric pressure (in hPa unit) = " +
                        str(current_pressure) +
            "\n humidity (in percentage) = " +
                        str(current_humidity) +
            "\n description = " +
                        str(weather_description))

    else:
        print(" City Not Found ")
        
    temperature = round (current_temperature - 273, 4)
    humidity = current_humidity
    pressure = current_pressure

    temperature_units = "oC"
    humidity_units = "%"
    pressure_units = "hPa"
    discharge_units = "m3s -1"
    

    ind_time = datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S.%f')
    print(ind_time)

    current_date = ind_time [:10]
    current_time = ind_time [11:-7]

    print ( f"Current Date, {current_date}" )
    print ( f"Current Time, {current_time}" )
    
    # compile and list down the data
    # We have Temperature, Humidity, Discharge, and Reference Image
    # make a dataframe,
    dischargeValue = discharge
    value_dict = {}
    value_dict [ "Date" ] = current_date
    value_dict [ "Time" ] = current_time
    value_dict ["State"] = "Himachal Pradesh"
    value_dict [ "City" ] = "Mandi"
    value_dict [ f"Temperature ({temperature_units}) " ] = f"{temperature}"
    value_dict [ f"Humidity ({humidity_units}) " ] = f"{humidity}"
    value_dict [ f"Discharge ({discharge_units}) " ] = f"{dischargeValue}"
    value_dict [ f"Pressure ({pressure_units})" ] = f"{pressure}"
    value_dict [ f"Weather Description" ] = f"{weather_description}"
    df = pd.DataFrame ( value_dict, index = [0] )
    print (df)

    reference_image = save_first_frame_as_numpy_array (video_path)
    plt.imshow ( reference_image )
    plt.show()
    
    return df, reference_image

In [7]:
def observation_append ( df, save_path, save_name ):
    save_format = ".xlsx"
    save_path = save_path.replace ( "\\" , "\\\\" )
    
    if save_name + save_format not in os.listdir ( save_path ):
        df.to_excel(save_path + "\\\\" + save_name + save_format)
    else:
        df1 = pd.read_excel ( save_path + "\\\\" + save_name + save_format, index_col = 0 )
        df1 = pd.concat([df1, df], ignore_index=True)
        df1.to_excel(save_path + "\\\\" + save_name + save_format)

In [8]:
def main ( video_path, video_name, save_path, save_name, Z, stream_width, state_name, city_name , set_initial_params ):
    format_ = ".mp4"
    video_path += "\\"
    video_path = video_path.replace ( "\\" , "\\\\" )
    video_path += video_name + format_
    if not set_initial_params:
        global xl , xr , yp , width , height , f_d
    else:
        xl , xr , yp , width , height , f_d = initial_alignment ( video_path, Z, stream_width )
    discharge = processing_and_discharge_calculation ( video_path, xl , xr , yp , \
                                                   width , height , f_d  )
    observation_df, reference_image = observation_get ( video_path, save_path, discharge,\
                                                       state_name, city_name )
    
    
    # now, to add to the excel
    observation_append ( observation_df, save_path, save_name )
    return observation_df , reference_image, xl , xr , yp , width , height , f_d

In [9]:
def send_email(sender_email, sender_password, receiver_email, subject, message, dataframe, image_array):
    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    
    # Attach the text message
    msg.attach(MIMEText(message, 'plain'))
    
    # Convert DataFrame to CSV and attach as file
    csv_data = dataframe.to_csv(index=False)
    part = MIMEBase('text', 'csv')
    part.set_payload(csv_data)
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment', filename='data.csv')
    msg.attach(part)
    
    # Convert NumPy array to image and attach
    image_data = image_array.astype(np.uint8)
    image = PIL.Image.fromarray(image_data)
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='JPEG')
    part = MIMEImage(image_bytes.getvalue())
    part.add_header('Content-Disposition', 'attachment', filename='image.jpg')
    msg.attach(part)
    
    # Send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        smtp.starttls()
        smtp.login(sender_email, sender_password)
        smtp.send_message(msg)

In [10]:
video_path = "D:\courses\IC201P design practicum"
video_name = "North Campus Stream"

save_path = "D:\courses\IC201P design practicum\Observations"
save_name = "observations"

# initial parameters
stream_width = 10.0 # width of the stream, in meters
Z = 30 # Distance of camera from the stream, in meters

state_name = "Himachal Pradesh"
city_name = "Mandi"

In [11]:
obs_df , reference_image, xl , xr , yp , width , height , f_d = \
main ( video_path, video_name, save_path, save_name, \
      Z, stream_width, state_name, city_name, set_initial_params = False )

NameError: name 'xl' is not defined

In [ ]:
print (obs_df)

plt.imshow(reference_image)
plt.show()

In [ ]:
# Provide the necessary details
def Send_mail():
    ind_time = datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S.%f')
    current_date = ind_time [:10]
    current_time = ind_time [11:-7]

    
    sender_email = 'b21010@students.iitmandi.ac.in'
    reciever_email = ""
    #receiver_email = 'drpatient1809@gmail.com'
    subject = 'IC201P Design Practicum, Test email'
    message = f'Observations for {current_date} {current_time}'

    file = open ( "D:\\courses\\IC201P design practicum\\something.txt" , "r" )
    sender_password = file.readline().strip()

    # Call the send_email function
    send_email(\
               sender_email, \
               sender_password, \
               receiver_email, \
               subject, \
               message, \
               dataframe = obs_df, \
               image_array = reference_image\
              )

In [ ]:
first_run = True
run_count = 1
import time

while( True ):
    
    obs_df , reference_image, xl , xr , yp , width , height , f_d = main ( video_path, video_name, save_path, save_name, Z, stream_width, state_name, city_name, set_initial_params = first_run)
    
    
    Send_mail()
    
    # parameters for the while loop
    first_run = False
    run_cout = run_count + 1
    # adding 10 seconds time delay
    time.sleep(10)